In [2]:
import torch
import torch.nn as nn
import torchvision
import pytorch_lightning as pl
from torch.nn import functional as F
import torchmetrics
import wandb
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split

In [3]:
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer

wandb_logger = WandbLogger(project="f5611")

wandb: Currently logged in as: pkantek. Use `wandb login --relogin` to force relogin


In [4]:
class CIFAR10DataModule(pl.LightningDataModule):
    def __init__(self, data_dir='../data', batch_size=64):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transform = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    def prepare_data(self):
        CIFAR10(self.data_dir, train=True, download=True)
        CIFAR10(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            cifar10 = CIFAR10(self.data_dir, train=True, transform=self.transform)
            self.cifar_train, self.cifar_val = random_split(cifar10, [45000, 5000])
        if stage == 'test' or stage is None:
            self.cifar_test = CIFAR10(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.cifar_train, batch_size=self.batch_size, num_workers=2)

    def val_dataloader(self):
        return DataLoader(self.cifar_val, batch_size=10 * self.batch_size, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.cifar_test, batch_size=10 * self.batch_size, num_workers=2)

In [5]:
def get_acc(n_classes):
    if n_classes > 2:
        return torchmetrics.Accuracy(task="multiclass", num_classes=n_classes)
    return torchmetrics.Accuracy(task="bianry")

# Define the PyTorch Lightning model
class CIFAR10Model(pl.LightningModule):
    def __init__(self, model, in_dims, n_classes=10, lr=1e-4):
        super().__init__()
        self.model = model
        
        self.save_hyperparameters(ignore=['model'])
        
        self.train_acc = get_acc(n_classes)
        self.valid_acc = get_acc(n_classes)
        self.test_acc = get_acc(n_classes)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits, loss = self.loss(x, y)
        preds = torch.argmax(logits, 1)
    
        self.log('train/loss', loss, on_epoch=True)
        self.train_acc(preds, y)
        self.log('train/acc', self.train_acc, on_epoch=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits, loss = self.loss(x, y)
        preds = torch.argmax(logits, 1)

        self.valid_acc(preds, y)
        self.log("valid/loss_epoch", loss)  # default on val/test is on_epoch only
        self.log('valid/acc_epoch', self.valid_acc)

        return logits

    def validation_epoch_end(self, outputs):
        dummy_input = torch.zeros(self.hparams["in_dims"], device=self.device)
        model_filename = f"model_{str(self.global_step).zfill(5)}.onnx"
        torch.onnx.export(self, dummy_input, model_filename, opset_version=11)
        artifact = wandb.Artifact(name="model.ckpt", type="model")
        artifact.add_file(model_filename)
        self.logger.experiment.log_artifact(artifact)

        flattened_logits = torch.flatten(torch.cat(validation_step_outputs))
        self.logger.experiment.log(
            {"valid/logits": wandb.Histogram(flattened_logits.to("cpu")),
             "global_step": self.global_step})
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits, loss = self.loss(x, y)
        preds = torch.argmax(logits, 1)

        self.test_acc(preds, y)
        self.log("test/loss_epoch", loss, on_step=False, on_epoch=True)
        self.log("test/acc_epoch", self.test_acc, on_step=False, on_epoch=True)
    
    def test_epoch_end(self, test_step_outputs):  # args are defined as part of pl API
        dummy_input = torch.zeros(self.hparams["in_dims"], device=self.device)
        model_filename = "model_final.onnx"
        self.to_onnx(model_filename, dummy_input, export_params=True)
        artifact = wandb.Artifact(name="model.ckpt", type="model")
        artifact.add_file(model_filename)
        wandb.log_artifact(artifact)
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits, loss = self.loss(x, y)
        preds = torch.argmax(logits, 1)

        self.valid_acc(preds, y)
        self.log("valid/loss_epoch", loss)
        self.log('valid/acc_epoch', self.valid_acc)

        return logits

    def validation_epoch_end(self, validation_step_outputs):
        dummy_input = torch.zeros(self.hparams["in_dims"], device=self.device)
        model_filename = f"model_{str(self.global_step).zfill(5)}.onnx"
        torch.onnx.export(self, dummy_input, model_filename, opset_version=11)
        artifact = wandb.Artifact(name="model.ckpt", type="model")
        artifact.add_file(model_filename)
        self.logger.experiment.log_artifact(artifact)

        flattened_logits = torch.flatten(torch.cat(validation_step_outputs))
        self.logger.experiment.log(
            {"valid/logits": wandb.Histogram(flattened_logits.to("cpu")),
             "global_step": self.global_step})

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams["lr"])
    
    def loss(self, x, y):
        logits = self(x)
        cel = torch.nn.CrossEntropyLoss()
        loss = cel(logits, y)
        return logits, loss

In [6]:
class ImagePredictionLogger(pl.Callback):
    def __init__(self, val_samples, num_samples=32):
        super().__init__()
        self.val_imgs, self.val_labels = val_samples
        self.val_imgs = self.val_imgs[:num_samples]
        self.val_labels = self.val_labels[:num_samples]
          
    def on_validation_epoch_end(self, trainer, pl_module):
        val_imgs = self.val_imgs.to(device=pl_module.device)

        logits = pl_module(val_imgs)
        preds = torch.argmax(logits, 1)

        trainer.logger.experiment.log({
            "examples": [wandb.Image(x, caption=self.labels_to_caption(pred, y)) 
                            for x, pred, y in zip(val_imgs, preds, self.val_labels)],
            "global_step": trainer.global_step
            })
        
    def labels_to_caption(self, pred, y):
        mapping = {0: "airplane", 1:"car", 2:"bird", 3:"cat", 4:"deer", 5:"dog", 6:"frog", 7:"horse", 8:"ship", 9: "truck"}
        pred_name = mapping[pred.item()]
        y_name = mapping[y.item()]
        return f"Pred:{pred_name}, Label:{y_name}"

In [7]:
mobilenet = torchvision.models.mobilenet_v3_large(weights=torchvision.models.MobileNet_V3_Large_Weights.IMAGENET1K_V2, progress=True)
mobilenet.classifier[3] = nn.Linear(in_features=1280, out_features=10, bias=True)
mobilenet.train()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bi

In [8]:
# setup data
cifar = CIFAR10DataModule()
cifar.prepare_data()
cifar.setup()

# grab samples to log predictions on
samples = next(iter(cifar.val_dataloader()))

Files already downloaded and verified
Files already downloaded and verified


In [12]:
trainer = pl.Trainer(
    logger=wandb_logger,    # W&B integration
    log_every_n_steps=50,   # set the logging frequency
    max_epochs=5,           # number of epochs
    deterministic=True,     # keep it deterministic
    callbacks=[ImagePredictionLogger(samples)] # see Callbacks section
    )

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
# setup model
model = CIFAR10Model(model=mobilenet, in_dims=(1, 3, 32, 32))

# fit the model
trainer.fit(model, cifar)

# evaluate the model on a test set
trainer.test(datamodule=cifar,
             ckpt_path=None)  # uses last-saved model

wandb.finish()

C:\Users\pkantek\AppData\Local\pypoetry\Cache\virtualenvs\ml-astr-wIJ34D6o-py3.10\lib\site-packages\pytorch_lightning\utilities\parsing.py:262: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(


Files already downloaded and verified
Files already downloaded and verified



  | Name      | Type               | Params
-------------------------------------------------
0 | model     | MobileNetV3        | 4.2 M 
1 | train_acc | MulticlassAccuracy | 0     
2 | valid_acc | MulticlassAccuracy | 0     
3 | test_acc  | MulticlassAccuracy | 0     
-------------------------------------------------
4.2 M     Trainable params
0         Non-trainable params
4.2 M     Total params
16.859    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

(1, 3, 32, 32)


...

Files already downloaded and verified
Files already downloaded and verified


Restoring states from the checkpoint path at .\f5611\20ew4paf\checkpoints\epoch=4-step=3520.ckpt
Loaded model weights from checkpoint at .\f5611\20ew4paf\checkpoints\epoch=4-step=3520.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test/acc_epoch       │    0.6718000173568726     │
│      test/loss_epoch      │    1.1043251752853394     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████
global_step,▁▁▂▂▄▄▅▅▇▇██
test/acc_epoch,▁
test/loss_epoch,▁
train/acc_epoch,▁▄▆▇█
train/acc_step,▁▁▂▄▄▅▄▄▄▅▅▆▆▆▅▅▆▆▆▆▆▆▆▆▇▅▇▇▇██▆▇▇██▇███
train/loss_epoch,█▅▃▂▁
train/loss_step,█▇▇▅▅▄▅▄▄▄▄▃▄▃▄▄▃▂▃▂▃▃▂▃▂▃▂▂▂▁▁▂▂▂▁▁▂▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
valid/acc_epoch,▁▆▇██
valid/loss_epoch,█▃▁▂▄


In [15]:
wandb.finish()

# Hyperparameter tuning

In [10]:
import ray


cifar_ref = ray.put(cifar)

In [10]:
from ray import air, tune
from ray.tune.integration.pytorch_lightning import TuneReportCallback

from ray.air import session
from ray.air.integrations.wandb import WandbLoggerCallback


def train_model(config):
    
    # setup model
    model = CIFAR10Model(model=mobilenet, in_dims=(1, 3, 32, 32), lr=config["lr"])
    
    # Create the Tune Reporting Callback
    metrics = {"loss": "valid/loss_epoch", "acc": "valid/acc_epoch"}
    # callbacks = [TuneReportCheckpointCallback(metrics, filename="checkpoint", on="validation_end")]

    trainer = pl.Trainer(
        # logger=wandb_logger,    # W&B integration
        log_every_n_steps=50,   # set the logging frequency
        max_epochs=5,           # number of epochs
        deterministic=True,     # keep it deterministic
        # callbacks=callbacks # see Callbacks section
    )
    
    # fit the model
    trainer.fit(model, ray.get(cifar_ref))


config = {
    "lr": tune.loguniform(1e-4, 1e-1)
}

# Make sure to specify how many actors each training run will create via the "extra_cpu" field.
tuner = tune.Tuner(
    train_model,
    tune_config=tune.TuneConfig(
        metric="loss",
        mode="min",
        num_samples=4
    ),
    run_config=air.RunConfig(
        name="tune_cifar",
            # callbacks=[
            #     WandbLoggerCallback(project="f5611")
            # ]
        ),
    param_space=config
)

results = tuner.fit()

print("Best hyperparameters found were: ", results.get_best_result().config)

 20%|█▉        | 33619968/170498071 [00:33<02:18, 991462.23it/s] 


 20%|██        | 34766848/170498071 [00:34<02:16, 991128.62it/s]
2023-01-15 10:36:13,154	WARNING worker.py:1851 -- Warning: The actor ImplicitFunc is very large (16 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(train_model pid=17920) GPU available: False, used: False
(train_model pid=17920) TPU available: False, using: 0 TPU cores
(train_model pid=17920) IPU available: False, using: 0 IPUs
(train_model pid=17920) HPU available: False, using: 0 HPUs


(train_model pid=17920) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ../data\cifar-10-python.tar.gz


  3%|▎         | 5144576/170498071 [00:04<03:02, 904752.45it/s]
(train_model pid=17832) GPU available: False, used: False
  3%|▎         | 5242880/170498071 [00:05<03:00, 915694.57it/s]
(train_model pid=17832) TPU available: False, using: 0 TPU cores
(train_model pid=17832) IPU available: False, using: 0 IPUs
(train_model pid=17832) HPU available: False, using: 0 HPUs
  3%|▎         | 5963776/170498071 [00:05<02:58, 924267.33it/s]


(train_model pid=17832) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to ../data\cifar-10-python.tar.gz


...

(train_model pid=18168) Extracting ../data\cifar-10-python.tar.gz to ../data
(train_model pid=18168) Files already downloaded and verified


(train_model pid=18168) Missing logger folder: C:\Users\pkantek\ray_results\tune_cifar\train_model_0c049_00003_3_lr=0.0715_2023-01-15_10-36-30\lightning_logs
(train_model pid=18168) 
(train_model pid=18168)   | Name      | Type               | Params
(train_model pid=18168) -------------------------------------------------
(train_model pid=18168) 0 | model     | MobileNetV3        | 4.2 M 
(train_model pid=18168) 1 | train_acc | MulticlassAccuracy | 0     
(train_model pid=18168) 2 | valid_acc | MulticlassAccuracy | 0     
(train_model pid=18168) 3 | test_acc  | MulticlassAccuracy | 0     
(train_model pid=18168) -------------------------------------------------
(train_model pid=18168) 4.2 M     Trainable params
(train_model pid=18168) 0         Non-trainable params
(train_model pid=18168) 4.2 M     Total params
(train_model pid=18168) 16.859    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]m 
Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:02<00:00,  1.43s/it]


2023-01-15 10:46:58,735	ERROR trial_runner.py:1088 -- Trial train_model_0c049_00003: Error processing event.
ray.exceptions.RayTaskError(AttributeError): ray::ImplicitFunc.train() (pid=18168, ip=127.0.0.1, repr=train_model)
  File "python\ray\_raylet.pyx", line 830, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 834, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 780, in ray._raylet.execute_task.function_executor
  File "C:\Users\pkantek\AppData\Local\pypoetry\Cache\virtualenvs\ml-astr-wIJ34D6o-py3.10\lib\site-packages\ray\_private\function_manager.py", line 674, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "C:\Users\pkantek\AppData\Local\pypoetry\Cache\virtualenvs\ml-astr-wIJ34D6o-py3.10\lib\site-packages\ray\util\tracing\tracing_helper.py", line 466, in _resume_span
    return method(self, *_args, **_kwargs)
  File "C:\Users\pkantek\AppData\Local\pypoetry\Cache\virtualenvs\ml-astr-wIJ34D6o-py3.10\lib\site-packag

RuntimeError: No best trial found for the given metric: loss. This means that no trial has reported this metric, or all values reported for this metric are NaN. To not ignore NaN values, you can set the `filter_nan_and_inf` arg to False.

In [ ]:

wandb.finish()

In [12]:
results.get_best_result().config

2023-01-15 10:51:05,799	WARNING experiment_analysis.py:627 -- Could not find best trial. Did you pass the correct `metric` parameter?


RuntimeError: No best trial found for the given metric: loss. This means that no trial has reported this metric, or all values reported for this metric are NaN. To not ignore NaN values, you can set the `filter_nan_and_inf` arg to False.

In [ ]:
from ray import air, tune
from ray.tune.integration.pytorch_lightning import TuneReportCallback

from ray.air import session
from ray.air.integrations.wandb import WandbLoggerCallback


def train_model(config):
    
    # setup model
    model = CIFAR10Model(model=mobilenet, in_dims=(1, 3, 32, 32), lr=config["lr"])
    
    # Create the Tune Reporting Callback
    metrics = {"loss": "valid/loss_epoch", "acc": "valid/acc_epoch"}
    callbacks = [TuneReportCallback(metrics, on="validation_end")]

    trainer = pl.Trainer(
        logger=wandb_logger,    # W&B integration
        log_every_n_steps=50,   # set the logging frequency
        max_epochs=5,           # number of epochs
        deterministic=True,     # keep it deterministic
        callbacks=callbacks # see Callbacks section
    )
    
    # fit the model
    trainer.fit(model, ray.get(cifar_ref))


config = {
    "lr": tune.loguniform(1e-4, 1e-1)
}

# Make sure to specify how many actors each training run will create via the "extra_cpu" field.
tuner = tune.Tuner(
    train_model,
    tune_config=tune.TuneConfig(
        metric="loss",
        mode="min",
        num_samples=4
    ),
    run_config=air.RunConfig(
        name="tune_cifar",
            # callbacks=[
            #     WandbLoggerCallback(project="f5611")
            # ]
        ),
    param_space=config
)

results = tuner.fit()

print("Best hyperparameters found were: ", results.get_best_result().config)

2023-01-15 10:52:17,507	WARNING worker.py:1851 -- Warning: The actor ImplicitFunc is very large (16 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(train_model pid=17996) GPU available: False, used: False
(train_model pid=17996) TPU available: False, using: 0 TPU cores
(train_model pid=17996) IPU available: False, using: 0 IPUs
(train_model pid=17996) HPU available: False, using: 0 HPUs


(train_model pid=17996) Files already downloaded and verified
(train_model pid=17996) Files already downloaded and verified


(train_model pid=17996) 
(train_model pid=17996)   | Name      | Type               | Params
(train_model pid=17996) -------------------------------------------------
(train_model pid=17996) 0 | model     | MobileNetV3        | 4.2 M 
(train_model pid=17996) 1 | train_acc | MulticlassAccuracy | 0     
(train_model pid=17996) 2 | valid_acc | MulticlassAccuracy | 0     
(train_model pid=17996) 3 | test_acc  | MulticlassAccuracy | 0     
(train_model pid=17996) -------------------------------------------------
(train_model pid=17996) 4.2 M     Trainable params
(train_model pid=17996) 0         Non-trainable params
(train_model pid=17996) 4.2 M     Total params
(train_model pid=17996) 16.859    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]m 


(train_model pid=11576) GPU available: False, used: False
(train_model pid=11576) TPU available: False, using: 0 TPU cores
(train_model pid=11576) IPU available: False, using: 0 IPUs
(train_model pid=11576) HPU available: False, using: 0 HPUs
(train_model pid=9880) GPU available: False, used: False
(train_model pid=9880) TPU available: False, using: 0 TPU cores
(train_model pid=9880) IPU available: False, using: 0 IPUs
(train_model pid=9880) HPU available: False, using: 0 HPUs
(train_model pid=3208) GPU available: False, used: False
(train_model pid=3208) TPU available: False, using: 0 TPU cores
(train_model pid=3208) IPU available: False, using: 0 IPUs
(train_model pid=3208) HPU available: False, using: 0 HPUs


...

Epoch 0:  15%|█▍        | 105/712 [02:12<12:44,  1.26s/it, loss=2.3, v_num=3zln]


Traceback (most recent call last):
  File "C:\Users\pkantek\AppData\Local\Programs\Python\Python310\lib\multiprocessing\queues.py", line 239, in _feed
    reader_close()
  File "C:\Users\pkantek\AppData\Local\Programs\Python\Python310\lib\multiprocessing\connection.py", line 182, in close
    self._close()
  File "C:\Users\pkantek\AppData\Local\Programs\Python\Python310\lib\multiprocessing\connection.py", line 282, in _close
    _CloseHandle(self._handle)
OSError: [WinError 6] The handle is invalid


Epoch 0:  98%|█████████▊| 699/712 [11:14<00:12,  1.04it/s, loss=2.18, v_num=3zln]
(train_model pid=17996) 
Validation: 0it [00:00, ?it/s] 
Epoch 0:  97%|█████████▋| 693/712 [11:18<00:18,  1.02it/s, loss=1.24, v_num=3zln]
(train_model pid=11576) 
Validation: 0it [00:00, ?it/s] 
Epoch 0:  98%|█████████▊| 699/712 [11:26<00:12,  1.02it/s, loss=1.25, v_num=3zln]
(train_model pid=3208) 
Validation: 0it [00:00, ?it/s]0m 
Epoch 0:  99%|█████████▉| 704/712 [11:30<00:07,  1.02it/s, loss=1.27, v_num=3zln]
(train_model pid=9880) 
Validation: 0it [00:00, ?it/s]0m 
(train_model pid=17996) 
Validation DataLoader 0:   0%|          | 0/8 [00:00<?, ?it/s]
(train_model pid=17996) 
Epoch 0:  99%|█████████▉| 705/712 [11:55<00:07,  1.02s/it, loss=1.05, v_num=3zln]
(train_model pid=11576) 
Validation DataLoader 0:   0%|          | 0/8 [00:00<?, ?it/s]
(train_model pid=17996) 
Epoch 0:  99%|█████████▉| 706/712 [11:59<00:06,  1.02s/it, loss=1.05, v_num=3zln]
(train_model pid=11576) 
Epoch 0:  99%|█████████▉| 7

Trial name,acc,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,loss,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_model_4ad8d_00000,0.731,2023-01-15_11-17-24,False,,d05fa9b2ae3b4d7c9faa7a3c4bb629d2,CZW21I26Q1204,2,0.781569,127.0.0.1,17996,1502.53,730.487,1502.53,1673777844,0,,2,4ad8d_00000,0
train_model_4ad8d_00001,0.1044,2023-01-15_11-20-01,False,,21d4a33ad26943d290a97de7b3b6739a,CZW21I26Q1204,2,2.3081,127.0.0.1,11576,1650.59,882.606,1650.59,1673778001,0,,2,4ad8d_00001,0.00600004
train_model_4ad8d_00002,0.5954,2023-01-15_11-19-06,False,,095d905fca6c4f6db43419d76e077d37,CZW21I26Q1204,2,1.28863,127.0.0.1,9880,1595.47,817.467,1595.47,1673777946,0,,2,4ad8d_00002,0.0059979
train_model_4ad8d_00003,0.681,2023-01-15_11-17-32,False,,a0b336232d4d41ea857f88295e95a23c,CZW21I26Q1204,2,0.929202,127.0.0.1,3208,1500.58,727.697,1500.58,1673777852,0,,2,4ad8d_00003,0.00799966


Epoch 1:   0%|          | 0/712 [00:00<?, ?it/s, loss=1.05, v_num=3zln]          
(train_model pid=9880) 
Epoch 0:  99%|█████████▉| 708/712 [12:07<00:04,  1.03s/it, loss=1.27, v_num=3zln]
(train_model pid=11576) 
Epoch 0: 100%|██████████| 712/712 [12:08<00:00,  1.02s/it, loss=2.16, v_num=3zln]
(train_model pid=3208) 
Epoch 0: 100%|█████████▉| 710/712 [12:08<00:02,  1.03s/it, loss=1.27, v_num=3zln]
(train_model pid=9880) 
Epoch 1:   0%|          | 0/712 [00:00<?, ?it/s, loss=2.16, v_num=3zln]          
(train_model pid=3208) 
Epoch 0: 100%|█████████▉| 711/712 [12:10<00:01,  1.03s/it, loss=1.27, v_num=3zln]
(train_model pid=9880) 
Epoch 0: 100%|█████████▉| 710/712 [12:13<00:02,  1.03s/it, loss=1.27, v_num=3zln]
(train_model pid=3208) 
Epoch 0: 100%|██████████| 712/712 [12:13<00:00,  1.03s/it, loss=1.27, v_num=3zln]
(train_model pid=9880) 
Epoch 1:   0%|          | 0/712 [00:00<?, ?it/s, loss=1.27, v_num=3zln]          
(train_model pid=9880) 
Epoch 1:  87%|████████▋ | 618/712 [11:00<01:4

In [10]:
import ray

In [11]:
from ray import air, tune
from ray.tune.integration.pytorch_lightning import TuneReportCallback

from ray.air import session
from ray.air.integrations.wandb import WandbLoggerCallback


def train_model(config):
    
    # setup model
    model = CIFAR10Model(model=mobilenet, in_dims=(1, 3, 32, 32), lr=config["lr"])
    
    # Create the Tune Reporting Callback
    metrics = {"loss": "valid/loss_epoch", "acc": "valid/acc_epoch"}
    callbacks = [TuneReportCallback(metrics, on="validation_end")]

    trainer = pl.Trainer(
        logger=wandb_logger,    # W&B integration
        log_every_n_steps=100,   # set the logging frequency
        max_epochs=5,           # number of epochs
        deterministic=True,     # keep it deterministic
        callbacks=callbacks, # see Callbacks section
        enable_progress_bar=False
    )
    
    trainer.fit(model, ray.get(cifar_ref))


config = {
    "lr": tune.loguniform(1e-4, 1e-1)
}

tuner = tune.Tuner(
    train_model,
    tune_config=tune.TuneConfig(
        metric="loss",
        mode="min",
        num_samples=4
    ),
    run_config=air.RunConfig(
        name="tune_cifar",
            # callbacks=[
            #     WandbLoggerCallback(project="f5611")
            # ]
        ),
    param_space=config
)

results = tuner.fit()

print("Best hyperparameters found were: ", results.get_best_result().config)

2023-01-15 11:56:44,759	WARNING worker.py:1851 -- Warning: The actor ImplicitFunc is very large (16 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(train_model pid=16996) GPU available: False, used: False
(train_model pid=16996) TPU available: False, using: 0 TPU cores
(train_model pid=16996) IPU available: False, using: 0 IPUs
(train_model pid=16996) HPU available: False, using: 0 HPUs


(train_model pid=16996) Files already downloaded and verified
(train_model pid=16996) Files already downloaded and verified


(train_model pid=16996) 
(train_model pid=16996)   | Name      | Type               | Params
(train_model pid=16996) -------------------------------------------------
(train_model pid=16996) 0 | model     | MobileNetV3        | 4.2 M 
(train_model pid=16996) 1 | train_acc | MulticlassAccuracy | 0     
(train_model pid=16996) 2 | valid_acc | MulticlassAccuracy | 0     
(train_model pid=16996) 3 | test_acc  | MulticlassAccuracy | 0     
(train_model pid=16996) -------------------------------------------------
(train_model pid=16996) 4.2 M     Trainable params
(train_model pid=16996) 0         Non-trainable params
(train_model pid=16996) 4.2 M     Total params
(train_model pid=16996) 16.859    Total estimated model params size (MB)
(train_model pid=16996) C:\Users\pkantek\AppData\Local\pypoetry\Cache\virtualenvs\ml-astr-wIJ34D6o-py3.10\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many wor

(train_model pid=6772) Files already downloaded and verified


(train_model pid=3248) GPU available: False, used: False
(train_model pid=3248) TPU available: False, using: 0 TPU cores
(train_model pid=3248) IPU available: False, using: 0 IPUs
(train_model pid=3248) HPU available: False, using: 0 HPUs


(train_model pid=12768) Files already downloaded and verified
(train_model pid=3248) Files already downloaded and verified
(train_model pid=6772) Files already downloaded and verified
(train_model pid=12768) Files already downloaded and verified
(train_model pid=3248) Files already downloaded and verified


(train_model pid=6772) 
(train_model pid=6772)   | Name      | Type               | Params
(train_model pid=6772) -------------------------------------------------
(train_model pid=6772) 0 | model     | MobileNetV3        | 4.2 M 
(train_model pid=6772) 1 | train_acc | MulticlassAccuracy | 0     
(train_model pid=6772) 2 | valid_acc | MulticlassAccuracy | 0     
(train_model pid=6772) 3 | test_acc  | MulticlassAccuracy | 0     
(train_model pid=6772) -------------------------------------------------
(train_model pid=6772) 4.2 M     Trainable params
(train_model pid=6772) 0         Non-trainable params
(train_model pid=6772) 4.2 M     Total params
(train_model pid=6772) 16.859    Total estimated model params size (MB)
(train_model pid=6772) C:\Users\pkantek\AppData\Local\pypoetry\Cache\virtualenvs\ml-astr-wIJ34D6o-py3.10\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which ma

Trial name,acc,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_model_4bb8d_00000,0.623,2023-01-15_12-57-09,True,,28551eaaf4914acd84627ca244c087da,0_lr=0.0052,CZW21I26Q1204,5,1.08385,127.0.0.1,16996,3619.9,699.474,3619.9,1673783829,0,,5,4bb8d_00000,0
train_model_4bb8d_00001,0.0982,2023-01-15_13-01-03,True,,c6571711bfae495f9dde9956facec392,1_lr=0.0277,CZW21I26Q1204,5,2.35855,127.0.0.1,6772,3847.4,711.435,3847.4,1673784063,0,,5,4bb8d_00001,0.00299931
train_model_4bb8d_00002,0.09,2023-01-15_13-01-06,True,,b489e2df2f8d4f06872eb3690bb37e01,2_lr=0.0688,CZW21I26Q1204,5,2.31379,127.0.0.1,12768,3850.99,711.809,3850.99,1673784066,0,,5,4bb8d_00002,0.00499988
train_model_4bb8d_00003,0.7638,2023-01-15_12-53-13,True,,e6c561fcefc74cd197f847112da816b8,3_lr=0.0005,CZW21I26Q1204,5,0.840888,127.0.0.1,3248,3376.87,632.425,3376.87,1673783593,0,,5,4bb8d_00003,0.00399852


(train_model pid=3248) `Trainer.fit` stopped: `max_epochs=5` reached.
(train_model pid=16996) `Trainer.fit` stopped: `max_epochs=5` reached.
(train_model pid=6772) `Trainer.fit` stopped: `max_epochs=5` reached.
(train_model pid=12768) `Trainer.fit` stopped: `max_epochs=5` reached.
2023-01-15 13:01:08,035	INFO tune.py:762 -- Total run time: 3865.49 seconds (3865.08 seconds for the tuning loop).


Best hyperparameters found were:  {'lr': 0.0005211356069429316}


In [16]:
results.get_dataframe()

,loss,acc,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,date,...,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,warmup_time,config/lr,logdir
0,1.083846,0.6230,699.474198,False,NaN,NaN,5,4bb8d_00000,28551eaaf4914acd84627ca244c087da,2023-01-15_12-57-09,...,3619.897151,16996,CZW21I26Q1204,127.0.0.1,3619.897151,0,5,0.000000,0.005161,C:\Users\pkantek\ray_results\tune_cifar\train_...
1,2.358554,0.0982,711.434555,False,NaN,NaN,5,4bb8d_00001,c6571711bfae495f9dde9956facec392,2023-01-15_13-01-03,...,3847.400492,6772,CZW21I26Q1204,127.0.0.1,3847.400492,0,5,0.002999,0.027726,C:\Users\pkantek\ray_results\tune_cifar\train_...
2,2.313794,0.0900,711.809440,False,NaN,NaN,5,4bb8d_00002,b489e2df2f8d4f06872eb3690bb37e01,2023-01-15_13-01-06,...,3850.991401,12768,CZW21I26Q1204,127.0.0.1,3850.991401,0,5,0.005000,0.068803,C:\Users\pkantek\ray_results\tune_cifar\train_...
3,0.840888,0.7638,632.424868,False,NaN,NaN,5,4bb8d_00003,e6c561fcefc74cd197f847112da816b8,2023-01-15_12-53-13,...,3376.872896,3248,CZW21I26Q1204,127.0.0.1,3376.872896,0,5,0.003999,0.000521,C:\Users\pkantek\ray_results\tune_cifar\train_...
